In [35]:
import pandas as pd
import numpy as np
import datetime as dt

# Read in the market variables
df = pd.read_pickle("China Daily Data.pickle")

# Calculate the 10d & 20d SMA of FXI and include as columns
df["FXI_SMA_10"] = df["FXI"].rolling(10).mean()
df["FXI_SMA_20"] = df["FXI"].rolling(20).mean()

# Calculate current FXI vs 10d & 20d SMA respectively and include as columns
df["FXI_vs_FXI_SMA_10"] = df["FXI"]/df["FXI_SMA_10"]
df["FXI_vs_FXI_SMA_20"] = df["FXI"]/df["FXI_SMA_20"]

# Calculate and include the slopes from linear regressions of FXI_vs_SMA_10 over 5d & 10d periods
df["FXI_vs_FXI_SMA_10_Slope_5"] = df["FXI_vs_FXI_SMA_10"].rolling(5).apply(lambda x: np.polyfit(range(len(x)), x, 1)[0])
df["FXI_vs_FXI_SMA_10_Slope_10"] = df["FXI_vs_FXI_SMA_10"].rolling(10).apply(lambda x: np.polyfit(range(len(x)), x, 1)[0])

# Calculate and include the slopes from linear regressions of FXI_vs_SMA_20 over 25d & 50d periods
df["FXI_vs_FXI_SMA_20_Slope_25"] = df["FXI_vs_FXI_SMA_20"].rolling(25).apply(lambda x: np.polyfit(range(len(x)), x, 1)[0])
df["FXI_vs_FXI_SMA_20_Slope_50"] = df["FXI_vs_FXI_SMA_20"].rolling(50).apply(lambda x: np.polyfit(range(len(x)), x, 1)[0])

# Manual input of the triggers for each of these slopes
Trigger_Slope_5 = -0.0016
Trigger_Slope_10 = 0.0021
Trigger_Slope_25 = -0.0070
Trigger_Slope_50 = 0.0016
Trigger_FXI_vs_FXI_SMA_10_Upper = 1.0310
Trigger_FXI_vs_FXI_SMA_10_Lower = 0.9630

# Create a column of indicators for each slope indicating whether breached
df["Signal_Slope_5"] = df["FXI_vs_FXI_SMA_10_Slope_5"].apply(lambda x: 1 if x > Trigger_Slope_5 else 0)
df["Signal_Slope_10"] = df["FXI_vs_FXI_SMA_10_Slope_10"].apply(lambda x: 1 if x > Trigger_Slope_10 else 0)
df["Signal_Slope_25"] = df["FXI_vs_FXI_SMA_20_Slope_25"].apply(lambda x: 1 if x > Trigger_Slope_25 else 0)
df["Signal_Slope_50"] = df["FXI_vs_FXI_SMA_20_Slope_50"].apply(lambda x: 1 if x > Trigger_Slope_50 else 0)
df["Signal_FXI_vs_FXI_SMA_10_Upper"] = df["FXI_vs_FXI_SMA_10"].apply(lambda x: 0 if x < Trigger_FXI_vs_FXI_SMA_10_Upper else -3)
df["Signal_FXI_vs_FXI_SMA_10_Lower"] = df["FXI_vs_FXI_SMA_10"].apply(lambda x: 0 if x > Trigger_FXI_vs_FXI_SMA_10_Lower else +1)

# Sum the indicators for each day
Sum_Signals = df["Signal_Slope_5"] + df["Signal_Slope_10"] \
                + df["Signal_Slope_25"] + df["Signal_Slope_50"] \
                + df["Signal_FXI_vs_FXI_SMA_10_Upper"] + df["Signal_FXI_vs_FXI_SMA_10_Lower"]

# Signal variable is then 1 if sum slope indicators >= 2
df["Signal"] = Sum_Signals.apply(lambda x: 1 if x >= 2 else 0)

# Save the signals to an excel file
df.to_excel("China Daily Signals.xlsx")

# Alco save the signals to a pickle file
df.to_pickle("China Daily Signals.pickle")

In [5]:
# Set the start date and end date
startdate = dt.date(2008, 1, 1)
enddate = dt.date(2021, 9, 3)

# Filter for the required period
df = df[startdate : enddate]

In [30]:
sum(df.Signal)/len(df)

0.6265646731571627

In [31]:
sum(df.Signal_Slope_5)/len(df)

0.5841446453407511

In [32]:
sum(df.Signal_Slope_10)/len(df)

0.31525266573945293

In [33]:
sum(df.Signal_Slope_20)/len(df)

0.9809921186833566

In [34]:
sum(df.Signal_Slope_50)/len(df)

0.08623087621696801